# Building a Spam Filter with Naive Bayes

In this project, we're going to study the practical side of the Naive Bayes algorithm by building a spam filter for SMS messages.

Our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

We'll use dataset, which was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the <a href="https://archive.ics.uci.edu/ml/datasets/sms+spam+collection">The UCI Machine Learning Repository</a>. You can also download the dataset directly <a href="https://dq-content.s3.amazonaws.com/433/SMSSpamCollection">from this link</a>.


In [1]:
import pandas as pd
import re 

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
sms_spam.shape

(5572, 2)

In [2]:
sms_spam['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

Before creating it, it's very helpful to first think of a way of testing how well it works. Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

- A training set, which we'll use to "train" the computer how to classify messages.
- A test set, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing. The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages.
- The test set will have 1,114 messages.


Let's start by randomizing the entire dataset to ensure that spam and ham messages are spread properly throughout the dataset.

In [3]:
sms_spam.sample(frac=1, random_state=1)
sms_training = sms_spam.iloc[:4458].reset_index(drop=True)
sms_test = sms_spam.iloc[4458:].reset_index(drop=True)

# Find the percentage of spam and ham in both the training and the test set
print('Sms_training: \n', sms_training['Label'].value_counts(normalize=True)*100, '\n')
print('Sms_test: \n', sms_test['Label'].value_counts(normalize=True)*100, '\n')

Sms_training: 
 ham     86.496187
spam    13.503813
Name: Label, dtype: float64 

Sms_test: 
 ham     86.983842
spam    13.016158
Name: Label, dtype: float64 



## Data cleaning


Next step is to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. Right now, our training and test sets have this format:

<img src="https://dq-content.s3.amazonaws.com/433/cpgp_dataset_1.png" alt=" ">

To make the calculations easier, we want bring the data to this format:

<img src="https://dq-content.s3.amazonaws.com/433/cpgp_dataset_2.png" alt=" ">

Let's do it.

In [4]:
# training set before
sms_training.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# Del punctuation and transform every letter to lower case

sms_training['SMS'] = sms_training['SMS'].str.replace(r'\W', ' ').str.lower()
sms_training.head() # training set after

,Label,SMS
0,ham,go until jurong point crazy available only ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives aro...


So now we have to make vocabulary and create a list with all of the unique words that occur in the messages of our training set.

In [6]:
vocabulary_list = []
sms_training['SMS'] = sms_training['SMS'].str.split()

for sms in sms_training['SMS']:
    for word in sms:
        vocabulary_list.append(word)
        
vocabulary_set = set(vocabulary_list)
vocabulary = list(vocabulary_set)

In [7]:
len(vocabulary)

7813

It looks like there are 7,813 unique words in all the messages of our training set.
Now we build a dictionary that we'll then convert to the DataFrame we need.

In [8]:
word_counts_per_sms = {unique_word: [0] * len(sms_training['SMS']) for unique_word in vocabulary}

for i, sms in enumerate(sms_training['SMS']):
    for word in sms:
        word_counts_per_sms[word][i] += 1

In [9]:
word_counts = pd.DataFrame(data=word_counts_per_sms)

sms_training_clean = pd.concat([sms_training, word_counts], axis=1)
sms_training_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585236,01223585334,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,ü
0,ham,"[go, until, jurong, point, crazy, available, o...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[ok, lar, joking, wif, u, oni]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter.



## Calculating Constants First

The Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above we need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:

- P(Spam) and P(Ham)
- N<sub>Spam</sub>, N <sub>Ham</sub>, N<sub>Vocabulary</sub>

Remember that:
- NSpam is equal to the number of words in all the spam messages.
- NHam is equal to the number of words in all the non-spam messages.

We'll also use Laplace smoothing and set <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>&#x03B1;<!-- α --></mi>
  <mo>=</mo>
  <mn>1</mn>
</math>.

In [10]:
# P(Spam) and P(Ham)
p_spam = sms_training_clean['Label'].value_counts(normalize=1)['spam']
p_ham = sms_training_clean['Label'].value_counts(normalize=1)['ham']

# Isolating spam and ham messages
spam_messages = sms_training_clean[sms_training_clean['Label'] == 'spam']
ham_messages = sms_training_clean[sms_training_clean['Label'] == 'ham']

# N_Spam, N_ham, N_vocabulary
n_spam = spam_messages['SMS'].apply(len).sum()
n_ham = ham_messages['SMS'].apply(len).sum()
n_vocabulary = len(vocabulary)

# initiate alpha
alpha = 1


All these terms will have constant values in our equations for every new message.

How let's calculate the probability P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) for each word in our vocabulary.

In [11]:
p_spam_words = {unique_word: 0 for unique_word in vocabulary}
p_ham_words = {unique_word: 0 for unique_word in vocabulary}

for word in vocabulary:
    p_word_given_spam = (spam_messages[word].sum() + alpha) / (n_spam + alpha * n_vocabulary)
    p_spam_words[word] = p_word_given_spam
    
    p_word_given_ham = (ham_messages[word].sum() + alpha) / (n_ham + alpha * n_vocabulary)
    p_ham_words[word] = p_word_given_ham


## Classifying A New Message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter using these two equations:
\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}
\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

In [13]:
def classify(message):
    message = message.replace(r'\W', ' ').lower().split()
    
    p_spam_given_message = 1
    p_ham_given_message = 1
    for word in message:
        if word in p_spam_words:
            p_spam_given_message *= p_spam_words[word]
        if word in p_ham_words:
            p_ham_given_message *= p_ham_words[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
        
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [14]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 7.904810158987672e-18
P(Ham|message): 2.6411793525822926e-19
Label: Spam


In [15]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 9.630750386822645e-17
P(Ham|message): 3.789612110065227e-14
Label: Ham


## Spam filter accuracy testing

The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [21]:
def classify_sms_test(message):
    message = message.replace(r'\W', ' ').lower().split()
    
    p_spam_given_message = 1
    p_ham_given_message = 1
    for word in message:
        if word in p_spam_words:
            p_spam_given_message *= p_spam_words[word]
        if word in p_ham_words:
            p_ham_given_message *= p_ham_words[word]
            
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [22]:
sms_test['predicted'] = sms_test['SMS'].apply(classify_sms_test)
sms_test.head()

,Label,SMS,predicted
0,ham,Aight should I just plan to come up later toni...,ham
1,ham,Die... I accidentally deleted e msg i suppose ...,ham
2,spam,Welcome to UK-mobile-date this msg is FREE giv...,spam
3,ham,This is wishing you a great day. Moji told me ...,ham
4,ham,Thanks again for your reply today. When is ur ...,ham


Now, we'll write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

In [27]:
correct = 0
total = sms_test.shape[0]

for row in sms_test.iterrows():
    row = row[1]
    if row['Label'] ==  row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1063
Incorrect: 51
Accuracy: 0.9542190305206463


The accuracy is close to 95.42%, which is quite good. Our spam filter looked at 1,114 messages and classified 1,063 correctly.

## What's next?

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 95.42% on the test set we used, which is a pretty good result considering that our initial goal was an accuracy of over 80%.

Next steps include:

- Analyze the 51 messages that were classified incorrectly and try to figure out why the algorithm classified them incorrectly
- Make the filtering process more complex by making the algorithm sensitive to letter case